# Network inference under noisy channels

Under noiseless hardware, it seems that covariance methods reign supreme over entropic quantities by both computational complexity and performance. We hope to find regimes for which studying entropic quantities are advantageous. 

The intuition for examining noisy quantum channels is as follows. Covariance is a linear operator. It captures the linear relationship between two random variables. Mutual information also quantifies the amount of correlation between random quantities, but it assumes no linearity. Thus, I suspect when the linear formalism of quantum mechanics break down, the performance of covariance-based inference will decline, and perhaps more so than entropy-based methods.

In [18]:
import qnetti
import pennylane as qml
from pennylane import numpy as qnp
import qnetvo

print(qnetvo.__version__)
print(qml.__version__)

0.4.1
0.28.0


In [19]:
def decision_matrix(mat, atol=0.1):
    return qnp.where(qnp.abs(mat) > atol, 1, 0)


def matrix_distance(mat1, mat2):
    return qnp.linalg.norm(qnp.array(mat1) - qnp.abs(qnp.array(mat2)))


def characteristic_matrix_inference(prep_node, expected_mat, **kwargs):
    char_mat = qnetti.qubit_characteristic_matrix(prep_node, **kwargs)
    dec_mat = decision_matrix(char_mat)
    char_dist = matrix_distance(expected_mat, char_mat)

    print("characteristic matrix :\n", char_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", char_dist)


def covariance_matrix_inference(
    prep_node,
    num_qubits,
    expected_mat,
    meas_wires=None,
    step_size=0.1,
    num_steps=10,
    verbose=False,
    shots=None,
    qnode_kwargs={},
    atol=0.1,
):
    settings = qnp.random.rand(3 * num_qubits, requires_grad=True)
    cov_cost = qnetti.qubit_covariance_cost_fn(
        prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    )

    settings_list = [settings]
    cost_vals = [cov_cost(settings)]

    opt = qml.GradientDescentOptimizer(stepsize=step_size)
    for i in range(num_steps):
        settings, cost_val = opt.step_and_cost(cov_cost, settings)

        cost_vals += [cost_val]
        settings_list += [settings]

    min_id = qnp.argmin(cost_vals)
    min_cost = cost_vals[min_id]
    opt_settings = settings_list[min_id]

    cov_mat = qnetti.qubit_covariance_matrix_fn(
        prep_node, meas_wires=meas_wires, shots=shots, qnode_kwargs=qnode_kwargs
    )(opt_settings)
    dec_mat = decision_matrix(cov_mat, atol=atol)
    cov_dist = matrix_distance(expected_mat, cov_mat)

    print("covariance matrix :\n", cov_mat)
    print("decision matrix :\n", dec_mat)
    print("expected matrix :\n", expected_mat)
    print("distance to expected : ", cov_dist)

## 2-qubit

In [38]:
ghz2_mat = qnp.ones((2, 2))

qnp.random.seed(100)
def rot_bell_state(settings, wires):
    qml.ArbitraryUnitary(qnp.random.normal(scale=0.1, size=(15,)), wires=wires)


bell_state_prep_node = qnetvo.PrepareNode(wires=[0, 1], ansatz_fn=rot_bell_state)

In [39]:
characteristic_matrix_inference(bell_state_prep_node, ghz2_mat, num_steps=20, step_size=0.1,qnode_kwargs={})

characteristic matrix :
 [[0.06788139 0.0593137 ]
 [0.0593137  0.83918277]]
decision matrix :
 [[0 0]
 [0 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  1.632326165101115


In [40]:
covariance_matrix_inference(
    bell_state_prep_node,
    2,
    ghz2_mat,
    step_size=0.2,
    num_steps=20,
    verbose=False,
)

covariance matrix :
 [[0.99235104 0.17600161]
 [0.17600161 0.98785177]]
decision matrix :
 [[1 1]
 [1 1]]
expected matrix :
 [[1. 1.]
 [1. 1.]]
distance to expected :  1.165398117348909


## 5-qubit

In [45]:
ghz5_mat = qnp.ones((5, 5))

qnp.random.seed(100)
def rot_ghz5_state(settings, wires):
    qml.ArbitraryUnitary(qnp.random.normal(scale=0.5,size=(4**5-1)), wires=wires)


ghz5_state_prep_node = qnetvo.PrepareNode(wires=[0, 1, 2, 3, 4], ansatz_fn=rot_ghz5_state)

In [49]:
characteristic_matrix_inference(ghz5_state_prep_node, ghz5_mat, num_steps=30, step_size=0.1)

characteristic matrix :
 [[0.83628743 0.11851519 0.04924075 0.15437617 0.12037577]
 [0.11851519 0.86893442 0.06252241 0.10531382 0.0849201 ]
 [0.04924075 0.06252241 0.94775039 0.12110324 0.13229768]
 [0.15437617 0.10531382 0.12110324 0.88077946 0.09034042]
 [0.12037577 0.0849201  0.13229768 0.09034042 0.88863827]]
decision matrix :
 [[1 1 0 1 1]
 [1 1 0 1 0]
 [0 0 1 1 1]
 [1 1 1 1 0]
 [1 0 1 0 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  4.018951550218008


In [48]:
covariance_matrix_inference(
    ghz5_state_prep_node, 5, ghz5_mat, step_size=0.1, num_steps=10, verbose=False
)

covariance matrix :
 [[ 0.99225657 -0.15597477  0.08848139 -0.01979018 -0.13554809]
 [-0.15597477  0.99599216  0.19995416 -0.07729274 -0.04936062]
 [ 0.08848139  0.19995416  0.99556714  0.23532129 -0.02855872]
 [-0.01979018 -0.07729274  0.23532129  0.99814727  0.07977523]
 [-0.13554809 -0.04936062 -0.02855872  0.07977523  0.99995769]]
decision matrix :
 [[1 1 0 0 1]
 [1 1 1 0 0]
 [0 1 1 1 0]
 [0 0 1 1 0]
 [1 0 0 0 1]]
expected matrix :
 [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
distance to expected :  4.005447614608562
